# Face Detection -> Face Tracking

### 1. 카메라 인스턴스 생성

In [1]:
from jetbot import Camera # 카메라 관련 라이브러리
from jetbot import bgr8_to_jpeg # widget통해서 이미지 전송시 bgr type을 jpeg로 변환

camera = Camera.instance(width=720, height=720)

# 오류 뜨거나 (initialized)

In [2]:
# 1) 왼쪽 정지버튼 아이콘 -> 전체 커널 ShutDown
# 2) 내 JupyterLab에서 Kernel Restart (단축키 : 0, 0)
# 3) 카메라 인스턴스 생성!!

# 이래도 에러 -> 재부팅
# 그래도 에러 -> 카메라 접촉 불량

# 2. 카메라 프리뷰 생성

In [5]:
# 텍스트 상자, 슬라이드, 체크박스, 드롭다운, 버튼 등 시각화관련 위젯
import ipywidgets.widgets as widgets

# 원격으로 접속했을 때, 카메라 정보 받아오는 라이브러리
from IPython.display import display

detect_face = widgets.Image(width=300, height=300, format='jpeg')
display(detect_face)

Image(value=b'', format='jpeg', height='300', width='300')

# 3. Face Detect (얼굴 감지)

### HaarCascade 

### 학습된 모델을 가져오자!!
- https://github.com/opencv/opencv/tree/master/data/haarcascades
- 전체 파일 및 디렉터리를 가져올 필요는 없다
- git clone 대신에 curl 사용

- curl : url 통해서 데이터 가져오기

In [10]:
# curl 명령어
# -L : Redirect 따라가서 가져오기
# -O : 형식 그대로 가져오기
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  908k  100  908k    0     0   927k      0 --:--:-- --:--:-- --:--:-- 1963k


In [11]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

face_x = face_y = face_w = face_h = 0

# ROI 활용, 눈 검출!!

In [15]:
### 눈 관련 학습 모델 다운로드!!
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  333k  100  333k    0     0   539k      0 --:--:-- --:--:-- --:--:--  839k


In [16]:
# 눈 관련 cascade 모델 생성!!
# 눈 좌표도 초기화!!

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

eye_x = eye_y = eye_w = eye_h = 0

In [ ]:
# IT 직군은 무엇이 있을까?
# Data Science -> Modeling, Engineering
# Model : 직접 ML, DL 모델 설계
# Engineering : 내 서비스에 어떤 모델을 써야 될까?

In [17]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300)) # 학습된 데이터가 300 x 300 이므로
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray) # 이미지 내에서 얼굴 검출!!
    
    # 만약, 검출되었다면
    if len(faces) > 0:
        (face_x, face_y, face_w, face_h) = faces[0]
    
        # 사각형 그리자 rectangle(img, start, end, bgrcolor, thickness)
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 255, 0), 4)
        
        roi_region = gray[face_y : face_y + face_h, face_x : face_x + face_w]
        roi_color = frame[face_y : face_y + face_h, face_x : face_x + face_w]
        
        eyes = eye_cascade.detectMultiScale(roi_region)
        
        # 눈이 검출되었다면 (눈은 2개임...)
        for eye_x, eye_y, eye_w, eye_h in eyes:
            cv2.rectangle(roi_color, (eye_x, eye_y), (eye_x + eye_w, eye_y + eye_h), (255, 0, 0), 2)
    
    detect_face.value = bgr8_to_jpeg(frame)

KeyboardInterrupt: 

# 4. 추적

### ServoMotor 관련된 port 권한 부여
- sudo chmod 777 /dev/ttyTHS1

In [1]:
# ServoMotor 제어
from servoserial import ServoSerial

servo = ServoSerial()

serial Open!


WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [2]:
servo.Servo_serial_double_control(1, 2200, 2, 2700)

245
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x98\x00\n\x02\n\x8c\x00\n\xf5'


In [3]:
center_x = 2200
center_y = 2700

# 5. PID(비례-적분-미분)
- 카메라가 얼굴 탐지 영역의 중심점을 찾아가는 데 
- 어느정도로 움직여야 하는지 그 정도를 계산하는 라이브러리

In [5]:
import PID

xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

In [ ]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300)) # 학습된 데이터가 300 x 300 이므로
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray) # 이미지 내에서 얼굴 검출!!
    
    # 만약, 검출되었다면
    if len(faces) > 0:
        (face_x, face_y, face_w, face_h) = faces[0]
    
        # 사각형 그리자 rectangle(img, start, end, bgrcolor, thickness)
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 255, 0), 4)
        
        # PID 활용해서 카메라 서보모터가 얼굴을 따라가자!!
        
        # X 오프셋
        xservo_pid.SystemOutput = face_x + face_w / 2  # x축 중심점
        xservo_pid.SetStepSignal(150)
        xservo_pid.SetInertiaTime(0.01, 0.006) # 다음 반응까지의 시간
        
        target_x = int(center_x + xservo_pid.SystemOutput)
        
        # Y 오프셋
        yservo_pid.SystemOutput = face_y + face_h / 2  # y축 중심점
        yservo_pid.SetStepSignal(150)
        yservo_pid.SetInertiaTime(0.01, 0.006)
        
        target_y = int(center_y + yservo_pid.SystemOutput)
        
        servo.Servo_serial_double_control(1, target_x, 2, target_y)
        
        
        roi_region = gray[face_y : face_y + face_h, face_x : face_x + face_w]
        roi_color = frame[face_y : face_y + face_h, face_x : face_x + face_w]
        
        eyes = eye_cascade.detectMultiScale(roi_region)
        
        # 눈이 검출되었다면 (눈은 2개임...)
        for eye_x, eye_y, eye_w, eye_h in eyes:
            cv2.rectangle(roi_color, (eye_x, eye_y), (eye_x + eye_w, eye_y + eye_h), (255, 0, 0), 2)
    
    detect_face.value = bgr8_to_jpeg(frame)